<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"></ul></div>

In [103]:
from scipy.special import comb

In [104]:
def get_fail_prob( lcomb, nrs, perr ):
    a = []
    for nopen in lcomb:
        p=0.0
        for nsel in np.arange( nopen, nrs, -1 ):
            p+= comb( nopen, nsel) * perr**nsel * (1-perr)**(nopen-nsel)
        a += [p]
    return a


In [105]:
def get_not_pass_prob( lprob ):
    if len(lprob) == 1:
        return lprob[0]
    else:
        recp = get_not_pass_prob( lprob[1:])
        return lprob[0] + recp - lprob[0]*recp


In [106]:
lprob = get_fail_prob( [3,2,1,2,5,6,1], 2, 1e-6)
get_not_pass_prob( lprob )

3.0999940000041991e-17

$$
\phi(N,M,(x_1,x_2,\dots,x_n)) = N\ {}_MC_{x_1}\phi(N-1, M-x_1,(x_2,x_3,\dots,x_n)) 
$$

In [100]:
import itertools
NUM_SYMBOL = 64
NUM_CHIP = 8
NUM_PLANE = 2
NUM_SIB = NUM_SYMBOL // ( NUM_CHIP * 2 ) # number of SIB in LIB
NUM_BLK = 200
numcor = 2

bb_tbl = np.random.randn( 100,2 ) * 4
bb_tbl = bb_tbl.clip( 0, 5).astype(int)
bb_tbl = bb_tbl[ np.sum( bb_tbl, axis=1) > 0 ]


In [102]:
def get_bb_tbl_libb2( bb_mask_libb2, bb_tbl_sibb ):
    ''' Make bb_tbl_libb2 from bb_tbl_sibb considering BB selection based on bb_mask_libb2
        bb_mask_lib2 : True:select BB, False:select normal Blk. array(NUM_SIB,len_bb_sibb)
        bb_tbl_sib   : number of BB each Blk in SIBB has
    '''
    bb_tbl_libb2 = np.zeros_like( bb_mask_libb2 )
    bb_tbl_sibb_tmp = bb_tbl_sibb.copy()
    for sib in range(bb_mask_libb2.shape[0]):
        bb_tbl_libb2[sib] = bb_tbl_sibb_tmp
        bb_tbl_sibb_tmp -= bb_mask_libb2[sib]

    return bb_tbl_libb2


In [103]:
def get_pass_prob_libb2( bb_mask_libb2, bb_tbl_libb2 ):
    ''' Get Pass probability
    '''
    pass_prob_libb2 = np.empty_like( bb_mask_libb2, dtype=np.float )
    left_blk_num_libb2 = np.empty_like( bb_mask_libb2, dtype=int)
    left_blk_num_libb2[0] = NUM_BLK
    for i in range( 1, left_blk_num_libb2.shape[0] ):
        left_blk_num_libb2[i] = left_blk_num_libb2[i-1] -1
#    print( left_blk_num_libb2 )
    pass_prob_libb2[bb_mask_libb2] = bb_tbl_libb2[bb_mask_libb2] \
                                     / left_blk_num_libb2[bb_mask_libb2] # Select BB
    pass_prob_libb2[~bb_mask_libb2] = 1.0 - bb_tbl_libb2[~bb_mask_libb2] \
                                            / left_blk_num_libb2[~bb_mask_libb2] # Select nomarl Blk

    return pass_prob_libb2

In [104]:
def get_prob( bb_chips, bb_tbl, numcorbb ):
    ''' Get PASS probability
        bb_chips : list of chips including BB in SIB
        bb_tbl   : number of BB per plane for each BB chip
        numcorbb   : number of RS correctable error symbols
    '''
    probs = {}
    prob_sum = 0.0
    bb_tbl_sibb = bb_tbl[ bb_chips].ravel()
    bb_len_sibb = len(bb_tbl_sibb)
    bb_len_libb = bb_len_sibb * NUM_SIB
    for numbb in range(numcorbb+1):
        probs[numbb] = {}    
        for bbpos in itertools.combinations( range(bb_len_libb), numbb ):
            bb_mask_libb = np.zeros((bb_len_libb),dtype=np.bool)
            bb_mask_libb[list(bbpos)] = True
            bb_mask_libb2 = bb_mask_libb.reshape( NUM_SIB, bb_len_sibb )
            if np.any( bb_mask_libb2.sum(axis=0) > bb_tbl_sibb ):
                continue
            bb_tbl_libb2 = get_bb_tbl_libb2( bb_mask_libb2, bb_tbl_sibb )
            pass_prob_libb2 = get_pass_prob_libb2( bb_mask_libb2, bb_tbl_libb2 )

            pass_prob_prod = pass_prob_libb2.prod()
            probs[numbb][bbpos] = pass_prob_prod
            prob_sum += pass_prob_prod
    return prob_sum, probs


In [140]:
bb_tbl[2] = [2,0]
bb_tbl[4] = [0,0]
bb_chips = [ 2, 4 ]
numcorbb = 1

In [141]:
prob_sum, probs = get_prob( bb_chips, bb_tbl, numcorbb)

In [142]:
probs20 = probs

In [144]:
print( probs11[1], probs20[1] )

{(0,): 0.0049000000000000007, (3,): 0.0049000000000000007, (4,): 0.0049000000000000007, (7,): 0.0049000000000000007, (8,): 0.0049000000000000007, (11,): 0.0049000000000000007, (12,): 0.004899999999999999, (15,): 0.004899999999999999} {(0,): 0.0049246231155778896, (4,): 0.0049494949494949493, (8,): 0.0049746192893401018, (12,): 0.0049999999999999992}


In [6]:
%qtconsole